In [ ]:
!pip --quiet install MPRAlib

# Quality Metrics

This python notebook uses a MPRA dataset to generate some general quality metrics.

In [2]:
# Loading the MPRAlib library
from mpralib.mpradata import MPRAdata
# Loading other libraries
import pandas as pd
import numpy as np

# Load the data
mpradata = MPRAdata.from_file("../resources/IGVFFI0402JCOL.tsv.gz")


## Pearson Correlation

In [3]:
mpradata.barcode_threshold = 10

print(mpradata.pearson_correlation.flatten()[[1,2,5]].mean().round(3))

0.943


## Median barcodes per oligo

## Median barcodes per oligo

## Median barcodes per oligo

## Median barcodes per oligo

## Median barcodes per oligo

## Median barcodes per oligo